# Petrov Leonid 6133-010402D. Lab№2

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType
from pyspark.sql import SparkSession

In [3]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession\
    .builder\
    .getOrCreate()

In [4]:
app_name = "Lab2_Petrov_6133-010402D"
conf = SparkConf().setAppName(app_name).setMaster('local')
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by getOrCreate at <ipython-input-3-17e21111a080>:5 

In [6]:
sc = spark_session._sc
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
!hadoop fs -ls /

Found 7 items
-rwxr-xr-x   3 root root       3841 2021-12-14 09:31 /Untitled.ipynb
drwxr-xr-x   - mapr mapr          5 2021-12-14 08:03 /apps
drwxr-xr-x   - mapr mapr          0 2021-12-14 08:06 /opt
-rwxr-xr-x   3 root root         24 2021-12-14 10:51 /test.txt
drwxrwxrwx   - mapr mapr          0 2021-12-14 07:54 /tmp
drwxr-xr-x   - mapr mapr          2 2021-12-14 18:01 /user
drwxr-xr-x   - mapr mapr          1 2021-12-14 07:58 /var


In [8]:
# hdfs не работает 
test = sc.textFile("/test.txt")
test.take(10)

['asdfasdfasdfasdfasdfasdf']

In [9]:
!head ./data/programming-languages.csv

name,wikipedia_url
A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)
A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)
A-0 System,https://en.wikipedia.org/wiki/A-0_System
A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)
A++,https://en.wikipedia.org/wiki/A%2B%2B
ABAP,https://en.wikipedia.org/wiki/ABAP
ABC,https://en.wikipedia.org/wiki/ABC_(programming_language)
ABC ALGOL,https://en.wikipedia.org/wiki/ABC_ALGOL
ABSET,https://en.wikipedia.org/wiki/ABSET


## Задание
<p>Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы.</p><br>
<p>Получившийся отчёт сохранить в формате Apache Parquet.</p>

In [10]:
data = sc.textFile("file:///home/workspace/data/programming-languages.csv")
data.take(10)

['name,wikipedia_url',
 'A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)',
 'A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)',
 'A-0 System,https://en.wikipedia.org/wiki/A-0_System',
 'A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)',
 'A++,https://en.wikipedia.org/wiki/A%2B%2B',
 'ABAP,https://en.wikipedia.org/wiki/ABAP',
 'ABC,https://en.wikipedia.org/wiki/ABC_(programming_language)',
 'ABC ALGOL,https://en.wikipedia.org/wiki/ABC_ALGOL',
 'ABSET,https://en.wikipedia.org/wiki/ABSET']

In [11]:
sc = spark_session._sc
prog_lang_schema = StructType([
    StructField("name", StringType(), False),
    StructField("wikipedia_url", StringType(), False),
])

In [12]:
df_prog_langs = spark_session.read.csv("file:///home/workspace/data/programming-languages.csv", schema=prog_lang_schema)

In [13]:
df_prog_langs.head(3)

[Row(name='name', wikipedia_url='wikipedia_url'),
 Row(name='A# .NET', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(.NET)'),
 Row(name='A# (Axiom)', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(Axiom)')]

In [14]:
p_l_list = [str(row[0]) for row in df_prog_langs.select('name').collect()]

In [15]:
p_l_list[:5]

['name', 'A# .NET', 'A# (Axiom)', 'A-0 System', 'A+']

## Просмотрим содержимое xml файла

In [16]:
!head -n 3 ./data/posts_sample.xml

���<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" 

In [17]:
df_posts_xml = spark_session.read.format("xml").options(rowTag="row").load('file:///home/workspace/data/posts_sample.xml')

In [18]:
df_posts_xml.head()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUse

In [19]:
def define_language(row):
    language_tag = None
    for lang in p_l_list:
        if '<' + lang.lower() + '>' in row._Tags.lower():
            language_tag = lang
            break
    if language_tag is None:
        return None
    return (row._Id, language_tag)

In [20]:
def check_date_range(row, yearr):
    left_border = datetime(year=yearr, month=1, day=1)
    right_border = datetime(year=yearr, month=12, day=31)
    created_date = row._CreationDate
    return created_date >= left_border and created_date <= right_border

## Т.к. в задании указано, что необходимо выполнить построение данных с 2010 по 2019 годы, то сделаем это.
<p>Для этого используем цикл for по требуемым годам (2010, 2019)</p>

In [23]:
yearObj = {}
for year in range(2010, 2020):    
    yearObj[year] = df_posts_xml.rdd\
        .filter(lambda row: row._Tags is not None and check_date_range(row, year))\
        .map(define_language)\
        .filter(lambda row: row is not None)\
        .keyBy(lambda row: row[1])\
        .aggregateByKey(
            0,
            lambda acc, value: acc + 1,
            lambda acc1, acc2: acc1 + acc2,
        )\
        .sortBy(lambda row: row[1], ascending=False)\
        .toDF()
    yearObj[year] = yearObj[year].select(col('_1').alias('Programming_Language'), col('_2').alias(f'Num_of_posts_in_{str(year)}'))
    yearObj[year].show()

+--------------------+--------------------+
|Programming_Language|Num_of_posts_in_2010|
+--------------------+--------------------+
|                Java|                  52|
|          JavaScript|                  44|
|                 PHP|                  42|
|              Python|                  25|
|         Objective-C|                  22|
|                   C|                  20|
|                Ruby|                  11|
|              Delphi|                   7|
|                   R|                   3|
|                Bash|                   3|
|         AppleScript|                   3|
|                Perl|                   3|
|                  F#|                   2|
|             Haskell|                   2|
|               BASIC|                   1|
|                  Go|                   1|
|               Mouse|                   1|
|               XPath|                   1|
|        ActionScript|                   1|
|              MATLAB|          

# Запись результатов
<p>Запишем результаты так-же через цикл for</p>

In [27]:
arrayParqFiles = []
for year in yearObj.keys():
    yearObj[year].write.parquet(f"/Leon_Result/Top_Lang_Progr_{year}.parquet")
    parquetFile = spark_session.read.parquet(f"/Leon_Result/Top_Lang_Progr_{year}.parquet")
    parquetFile.createOrReplaceTempView(f"parquetFile_{year}")
    arrayParqFiles.append(parquetFile)
    print(f"Top 10 Prog Languages in {year}")
    spark_session.sql(f"SELECT Programming_Language FROM parquetFile_{year} ORDER BY Num_of_posts_in_{year} DESC LIMIT 10").show()

Top 10 Prog Languages in 2010
+--------------------+
|Programming_Language|
+--------------------+
|                Java|
|          JavaScript|
|                 PHP|
|              Python|
|         Objective-C|
|                   C|
|                Ruby|
|              Delphi|
|                Bash|
|         AppleScript|
+--------------------+

Top 10 Prog Languages in 2011
+--------------------+
|Programming_Language|
+--------------------+
|                 PHP|
|                Java|
|          JavaScript|
|              Python|
|         Objective-C|
|                   C|
|                Ruby|
|              Delphi|
|                Perl|
|                Bash|
+--------------------+

Top 10 Prog Languages in 2012
+--------------------+
|Programming_Language|
+--------------------+
|                 PHP|
|          JavaScript|
|                Java|
|              Python|
|         Objective-C|
|                   C|
|                Ruby|
|                Bash|
|          